#                            Stock Analysis and Price Prediction

# Contents
    1. Introduction
    2. Importing libraries
    3. Reading datasets
    4. Building Models
    5. Conclusion

# Introduction
In this project we will show how to write a python program that predicts the price of stocks using a machine learning technique called Long Short-Term Memory (LSTM) as well as create a optimize portfoilo using Efficient Frontier.

We will be solve the following question:

1. What was the change in price of the stock over time?
2. What was the daily return of the stock on average?
3. What was the moving average of the various stocks?
4. What was the correlation between different stocks'?
5. How much value do we put at risk by investing in a particular stock?
6. Portfoilo optimization using Efficient Frontier?
7. How can we attempt to predict future stock behavior using LSTM?

# Importing Libraries

In [ ]:
# here we are importing important libraries
import math
import pandas_datareader as pdr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM
from pandas_datareader import data as web
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

plt.style.use("fivethirtyeight")
sns.set_style('whitegrid')
%matplotlib inline

# Reading Dataset

Companies names and thier Ticker that we used for our Analysis
    
    1. Apple Inc. = AAPL
    2. Alphabet Inc. = GOOG
    3. Microsoft Corporation = MSFT
    4. Amazon = AMZN
    5. Facebook Inc. = FB
    6. Alibaba Group = BABA
    7. Johnson & Johnson = JNJ
    8. JPMorgon Chase & Co. = JPM
    9. ExxonMobil = XOM
    10.Bank of America = BAC
    11.WalMart Store Inc. = WMT
    12.Wells Fargo & Co. = WFC
    13.Visa Inc. = V
    14.Procter & Gamble Co. = PG
    15.Verizon Communication = VZ
    16.AT&T Inc. = T
    17.UnitedHealth Group Inc. = UNH
    18.Home Depot = HD
    19.Intel = INTC
    20.Oracle = ORCL


1. What was the change in price of the stock over time?

In [ ]:
#List of ticker of companies
Tech_list = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'FB', 'BABA','JNJ', 'JPM', 'XOM', 'BAC', 'WMT', 'WFC', 'V', 'PG', 'VZ', 'T', 'UNH', 'HD', 'INTC', 'ORCL']
data = pdr.get_data_yahoo(Tech_list, start = '2015-01-01')
data

In [ ]:
# Getting Monthly Adjused Close of all companies
aclose = data['Adj Close'].resample('M').ffill()
aclose.head()

In [ ]:
#Here is quick summary of each company Monthly Adjusted closing
aclose.describe()

In [ ]:
# Getting Volume Adjusted Close of all companies
volume = data['Volume'].resample('M').ffill()
volume.head()

2. What was the daily return of the stock on average?

In [ ]:
# Calculating monthly percentage change
monthly_returns = data['Adj Close'].resample('M').ffill().pct_change()
monthly_returns.tail()

In [ ]:
# here we are visualising of Monthly Adjusted Price, Monthly Volume and Monthly % Change

fig, axes = plt.subplots(nrows = 20, ncols = 3)
fig.suptitle('Monthly Adjusted Closing VS Monthly Volume VSMonthly % Change')
fig.set_figheight(80)
fig.set_figwidth(15)
#plt.subplots_adjust(top=1.25, bottom=1.2)

columns = list(aclose) 
  
for i, cols in enumerate(columns,0):
    aclose[cols].plot(ax = axes[i,0])
    axes[i,0].set(xlabel='Date', ylabel='Adj Close')
    axes[i,0].set_title(f"{Tech_list[i]}")
    volume[cols].plot(ax = axes[i,1])
    axes[i,1].set(xlabel='Date', ylabel='Volume')
    axes[i,1].set_title(f"{Tech_list[i]}")
    monthly_returns[cols].plot(ax = axes[i,2])
    axes[i,2].set(xlabel='Date', ylabel='Daily % change')
    axes[i,2].set_title(f"{Tech_list[i]}")

In [ ]:
avg = monthly_returns.mean()
plt.figure(figsize=(10, 5))
plt.scatter(columns, monthly_returns.mean())
plt.suptitle('Average Monthly Return')
plt.show()

In [ ]:
top_five = avg.nlargest(5)
company_name = list(top_five.keys())
company_name

In [ ]:
plt.figure(figsize=(7, 3))
plt.scatter(company_name, top_five)
plt.suptitle('Top 5 companies with highest Monthly Return')
plt.ylabel('Daily return Mean')
plt.xlabel('Name')
plt.show()

Now that we've brock down our companies from 20 to 5 on the basic of average percentage change in stock price and also seen the visualizations for the Adjusted price and the volume traded each day with daily percentage change, let's go ahead and caculate the moving average for the stock.

3. What was the moving average of the various stocks?

There are three important moving averages that we have applied to our charts so that it will help us to trade better. They are the 10 moving average, the 20 moving average and the 50 moving average. The 20 moving average (10MA) is the short-term outlook. The 50 moving average (20MA) is the medium term outlook. The 200 moving average (50MA) is the trend bias

In [ ]:
# now we are using for loop for grabing yahoo data and setting it in form of dataframe
#  Using globals() is a sloppy way of setting the DataFrame names, but its simple
for stock in company_name:
    globals()[stock] = web.DataReader(stock,"yahoo",'2015-01-01',datetime.now())


In [ ]:
company_list = [AMZN,AAPL,FB,MSFT,BABA]
for company, comp_name in zip(company_list,company_name):
    company["company_name"] = comp_name
    
df = pd.concat(company_list,axis=0)
df.tail(10)

In [ ]:
ma_day = [10, 20, 50]

for ma in ma_day:
    for company in company_list:
        column_name = f"MA for {ma} days"
        company[column_name] = company['Adj Close'].rolling(ma).mean()
        
company.head(15)

In [ ]:
# here we are visualising the additional moving averages
df.groupby("company_name").hist(figsize=(12, 12));

In [ ]:
AAPL.columns

In [ ]:
# here we are visualising three important moving averages of all the company
fig, axes = plt.subplots(nrows=3,ncols=2)
fig.set_figheight(20)
fig.set_figwidth(15)

AMZN[['Adj Close', 'MA for 10 days', 'MA for 20 days', 'MA for 50 days']].plot(ax=axes[0,0])
axes[0,0].set_title('AMAZON')

AAPL[['Adj Close', 'MA for 10 days', 'MA for 20 days', 'MA for 50 days']].plot(ax=axes[0,1])
axes[0,1].set_title('APPLE')

FB[['Adj Close', 'MA for 10 days', 'MA for 20 days', 'MA for 50 days']].plot(ax=axes[1,0])
axes[1,0].set_title('FACEBOOK')

MSFT[['Adj Close', 'MA for 10 days', 'MA for 20 days', 'MA for 50 days']].plot(ax=axes[1,1])
axes[1,1].set_title('MICROSOFT')

BABA[['Adj Close', 'MA for 10 days', 'MA for 20 days', 'MA for 50 days']].plot(ax=axes[2,0])
axes[2,0].set_title('UnitedHealth')

fig.tight_layout()

In [ ]:
# We'll use pct_change to find the percent change for each day
for company in company_list:
    company['Daily Return'] = company['Adj Close'].pct_change()

plt.figure(figsize=(12, 12))

for i, company in enumerate(company_list, 1):
    plt.subplot(3, 2, i)
    sns.distplot(company['Daily Return'].dropna(), bins=100, color='blue')
    plt.ylabel('Daily Return')
    plt.title(f'{company_name[i - 1]}')

4. What was the correlation between different stocks'?

In [ ]:
# Getting Top five company Adjusted Close price
aClose_five= aclose[company_name]
aClose_five.head()

In [ ]:
# here we are Making a new tech returns DataFrame
tech_rets = aClose_five.pct_change()
tech_rets.head()

In [ ]:
# here we are comparing Amazon to itself should show a perfectly linear relationship
sns.jointplot('AMZN', 'AMZN', tech_rets, kind='scatter', color = "blue")

In [ ]:
# here We'll use joinplot to compare the daily returns of Amazon and Microsoft
sns.jointplot('AMZN', 'MSFT', tech_rets, kind='scatter', color = "blue")

So now we can see that if two stocks are perfectly (and positivley) correlated with each other a linear relationship bewteen its daily return values should occur.

In [ ]:
# Here we are simply calling pairplot on our DataFrame for an automatic visual analysis 
# of all the comparisons
sns.pairplot(tech_rets, kind='reg')

In [ ]:
# Here we are using seabron for a quick correlation plot for the daily returns
sns.heatmap(tech_rets.corr(), annot=True, cmap="YlGnBu")

5. Portfoilo optimization using Efficient Frontier?

In [ ]:
tech_rets.head()

In [ ]:
#Analysing of Annual portfiolo return and risk assuming 20% weight on each stock
#assuming trading days = 252 days in a year

weights = np.array([0.2,0.2,0.2,0.2,0.2])

Preturn = np.sum(tech_rets.mean()* weights)*252
str(round((Preturn * 100),2))+'%'

In [ ]:
cov_matrix_annual = tech_rets.cov()*252
cov_matrix_annual

In [ ]:
Pvariance = np.dot(weights.T, np.dot(cov_matrix_annual,weights))
Pvariance

In [ ]:
Pstd = np.sqrt(Pvariance)
str(round((Pstd * 100),2))+'%'

In [ ]:
mu=expected_returns.mean_historical_return(aClose_five)
S= risk_models.sample_cov(aClose_five)

In [ ]:
mu

In [ ]:
S

In [ ]:
ef=EfficientFrontier(mu,S)
weights=ef.min_volatility()
cleaned_weights=ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

In [ ]:
ef=EfficientFrontier(mu,S)
weights=ef.max_sharpe()
cleaned_weights=ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

6. How much value do we put at risk by investing in a particular stock?

In [ ]:
#Here e are defining a new DataFrame as a cleaned version of the oriignal tech_rets DataFrame
rets = tech_rets.dropna()

area = np.pi*20

plt.figure(figsize=(7, 5))
plt.scatter(rets.mean(), rets.std(), s=area)
plt.xlabel('Expected return')
plt.ylabel('Risk')

for label, x, y in zip(rets.columns, rets.mean(), rets.std()):
    plt.annotate(label, xy=(x, y), xytext=(50, 50), textcoords='offset points', ha='right', va='bottom', 
                 arrowprops=dict(arrowstyle='-', color='blue', connectionstyle='arc3,rad=-0.3'))

7. How can we attempt to predict future stock behavior using LSTM?

In [ ]:
# Filtering the columns
df = MSFT.iloc[:,0:6]

In [ ]:
# here we are Visualising the closing price history
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.show()

Create a new data frame with only the closing price and convert it to an array. Then create a variable to store the length of the training data set. I want the training data set to contain about 80% of the data.

In [ ]:
#Creating a new dataframe with only the 'Close' column
data = df.filter(['Close'])
#Converting the dataframe to a numpy array
dataset = data.values
#Get /Compute the number of rows to train the model on
training_data_len = math.ceil( len(dataset) *.8)
training_data_len

Now scale the data set to be values between 0 and 1 inclusive, I do this because it is generally good practice to scale your data before giving it to the neural network.

In [ ]:
# here we are Scaling the all of the data to be values between 0 and 1 
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaled_data = scaler.fit_transform(dataset)
scaled_data

In [ ]:
#Creating the scaled training data set
train_data = scaled_data[0:training_data_len  , : ]
#Spliting the data into x_train and y_train data sets
x_train=[]
y_train = []
for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])

In [ ]:
#Here we are Converting x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
# Here we are reshaping the data into the shape accepted by the LSTM
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [ ]:
#now we are Building the LSTM network model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

In [ ]:
# here we are Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# here we are training the model
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
# here we are testing data set
test_data = scaled_data[training_data_len - 60: , : ]
#Creating the x_test and y_test data sets
x_test = []
y_test =  dataset[training_data_len : , : ] #Get all of the rows from index 1603 to the rest and all of the columns (in this case it's only column 'Close'), so 2003 - 1603 = 400 rows of data
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])

In [ ]:
# here we are converting x_test to a numpy array  
x_test = np.array(x_test)

In [ ]:
# here we are reshaping the data into the shape accepted by the LSTM  
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [ ]:
# now we are getting the models predicted price values
predictions = model.predict(x_test) 
predictions = scaler.inverse_transform(predictions)#Undo scaling

In [ ]:
# here we are calculaing the value of Root Mean Square Error
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
rmse

In [ ]:
#Plot/Create the data for the graph
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
#Visualize the data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
valid